# DID

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import gpboost as gpb
from patsy import dmatrix

In [64]:
filepath = '../../../../../data/processed/df_filtered_5years_exclude_too_far_islands_exclude_already_treatment.xlsx'
df = pd.read_excel(filepath)

# 特定の列の外れ値を除去する例
df['population_change_rate'] = df.groupby('island_id')['population'].pct_change() * 100
column = 'population_change_rate'

# 平均と標準偏差の計算
mean = df[column].mean()
std = df[column].std()

# 外れ値の定義（ここでは3標準偏差を超えるデータを外れ値とみなす）
lower_bound = mean - 3 * std
upper_bound = mean + 3 * std

# 外れ値を除去
df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# 除去されたデータを確認
outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
outliers

/tmp/ipykernel_14079/2430644254.py:5: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df['population_change_rate'] = df.groupby('island_id')['population'].pct_change() * 100


,island,year,island_id,region_code,region_name,prefecture_code,population,treatment_group,treatment_group_mainland,year_bridge,after_treatment,passage_of_year,year_connect_mainland,dummy_after_connect_mainland,income,prefecture_name,area_km2,distance_m,population_change_rate
190,因島,1995,94,34206,（旧）因島市,34,28411.0,1,1,1983,1,12,1983,1,3506.231862,広島県,35.04,800.0,121.804981
211,生口島,1995,95,34430,（旧）瀬戸田町,34,11097.0,1,1,1991,1,4,1991,1,1509.931029,広島県,31.21,500.0,87.007078
225,大久野島,2010,99,34203,竹原市,34,26.0,0,0,0,0,0,0,0,3387.313924,広島県,0.72,1263.0,73.333333
242,長島,1985,103,34427,（旧）大崎町,34,39.0,1,0,1987,0,0,0,0,NaN,広島県,1.04,350.0,-72.916667
364,江田島・能美島,1990,117,3431X,（旧）江田島町・大柿町・沖美町・能美町,34,37257.0,1,1,1973,1,17,1973,1,1807.199041,広島県,91.33,300.0,227.390158
380,沖野島,2015,118,34215,江田島市,34,9.0,1,1,1972,1,43,1973,1,2769.395700,広島県,0.94,30.0,125.000000
399,屋代島,2000,125,35302,（旧）大島町,35,21796.0,1,1,1976,1,24,1976,1,747.373507,山口県,128.48,950.0,171.499751


In [29]:
df_cleaned['log_income'] = np.log(df_cleaned['income'])
df_cleaned['log_area'] = np.log(df_cleaned['area_km2'])
df_cleaned['log_distance'] = np.log(df_cleaned['distance_m'])
df_cleaned = df_cleaned.dropna(subset=['population_change_rate', 'log_income'])

/tmp/ipykernel_14079/154425691.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['log_income'] = np.log(df_cleaned['income'])
/tmp/ipykernel_14079/154425691.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['log_area'] = np.log(df_cleaned['area_km2'])
/tmp/ipykernel_14079/154425691.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

## DID

In [65]:
y = df_cleaned['population_change_rate']
X = dmatrix('treatment_group * after_treatment', data=df_cleaned, return_type='dataframe')

model = sm.OLS(y, X)
result = model.fit()
print(result.summary())

                              OLS Regression Results                              
Dep. Variable:     population_change_rate   R-squared:                       0.045
Model:                                OLS   Adj. R-squared:                  0.042
Method:                     Least Squares   F-statistic:                     16.84
Date:                    Thu, 31 Oct 2024   Prob (F-statistic):           7.13e-08
Time:                            00:11:47   Log-Likelihood:                -2843.7
No. Observations:                     715   AIC:                             5693.
Df Residuals:                         712   BIC:                             5707.
Df Model:                               2                                         
Covariance Type:                nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

In [68]:
y = df_cleaned['population_change_rate']
group = df_cleaned['island_id']
X = dmatrix('treatment_group * after_treatment', data=df_cleaned, return_type='dataframe')

gp_model = gpb.GPModel(group_data=group, likelihood="gaussian")
gp_model.fit(y=y, X=X, params={'std_dev': True})
gp_model.summary()

[GPBoost] [Warning] The linear regression covariate data matrix (fixed effect) is rank deficient. This is not necessarily a problem when using gradient descent. If this is not desired, consider dropping some columns / covariates 
Model summary:
 Log-lik     AIC     BIC
 -2842.3 5696.59 5724.03
Nb. observations: 715
Nb. groups: 81 (island_id)
-----------------------------------------------------
Covariance parameters (random effects):
              Param.  Std. dev.
Error_term  160.6408     9.0026
island_id     6.2924     3.9491
-----------------------------------------------------
Linear regression coefficients (fixed effects):
                                  Param.  Std. dev.  z value  P(>|z|)
Intercept                       -15.5206        NaN      NaN      NaN
treatment_group                   6.2715        NaN      NaN      NaN
after_treatment                  -1.0956        inf     -0.0      1.0
treatment_group:after_treatment   0.0000        inf      0.0      1.0


## TWFE

In [70]:
y = df_cleaned['population_change_rate']
group = df_cleaned[['island_id']]
X = dmatrix('treatment_group + C(year) + C(passage_of_year) -1', data=df_cleaned, return_type='dataframe')

gp_model = gpb.GPModel(group_data=group, likelihood="gaussian")
gp_model.fit(y=y, X=X, params={'std_dev': True})
gp_model.summary()

Model summary:
 Log-lik     AIC     BIC
-2802.02 5726.05 6004.96
Nb. observations: 715
Nb. groups: 81 (island_id)
-----------------------------------------------------
Covariance parameters (random effects):
              Param.  Std. dev.
Error_term  142.2023     7.9712
island_id     7.4427     3.7939
-----------------------------------------------------
Linear regression coefficients (fixed effects):
                           Param.  Std. dev.  z value  P(>|z|)
C(year)[1975]            -19.5590     1.6812 -11.6339   0.0000
C(year)[1980]            -13.4971     1.6650  -8.1063   0.0000
C(year)[1985]            -12.1701     1.6544  -7.3564   0.0000
C(year)[1990]            -13.0365     1.6566  -7.8695   0.0000
C(year)[1995]            -20.4333     1.6628 -12.2884   0.0000
C(year)[2000]            -15.1889     1.5092 -10.0639   0.0000
C(year)[2005]            -10.0058     1.5240  -6.5655   0.0000
C(year)[2010]            -15.6171     1.5437 -10.1167   0.0000
C(year)[2015]            -1

## Staggered DiD

$$
Y_{it} = \text{Treatment}_i + \text{Year}_t + \text{Passage} \times \text{Cohort}
$$

In [81]:
y = df_cleaned['population_change_rate']
X = dmatrix('treatment_group + C(year) + C(passage_of_year):C(year_bridge) -1', data=df_cleaned, return_type='dataframe')

model = sm.OLS(y, X)
result = model.fit(cov_type='HC0')
print(result.summary())

interaction_terms = [col for col in X.columns if 'C(passage_of_year):C(year_bridge)' in col]
interaction_sum = result.params[interaction_terms].sum()

print(f"Sum of C(passage_of_year):C(year_bridge) parameters: {interaction_sum}")

                              OLS Regression Results                              
Dep. Variable:     population_change_rate   R-squared:                       0.306
Model:                                OLS   Adj. R-squared:                  0.097
Method:                     Least Squares   F-statistic:                       nan
Date:                    Thu, 31 Oct 2024   Prob (F-statistic):                nan
Time:                            00:42:27   Log-Likelihood:                -2729.9
No. Observations:                     715   AIC:                             5792.
Df Residuals:                         549   BIC:                             6551.
Df Model:                             165                                         
Covariance Type:                      HC0                                         
                                                     coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------